In [1]:
import tbapy
import numpy as np
tba = tbapy.TBA('70PhghWVToSpFoaK0scpRo6PCL7iKUff8tPhQXj4LrhQgwA60t7SBf5Tt4XvefzX')
year = 2019
output_file = "match_data.csv"
output_file2 = "match_simple.csv"
# 0 = RED
# 1 = BLUE

In [2]:
# event type guide: https://github.com/the-blue-alliance/the-blue-alliance/blob/master/consts/event_type.py#L2
all_events = sorted(list(filter(lambda e : e.event_type >= 0 and e.event_type <= 5, tba.events(year))), key = lambda a : a.start_date) # all regular season events
pre_champs_events = list(filter(lambda e : e.event_type not in [3, 4], all_events))
champs_events = list(filter(lambda e : e.event_type in [3, 4], all_events))

In [3]:
print(f"Total # Events: {len(all_events)}")
print(f"Pre-Champs (Training) Events: {len(pre_champs_events)}")
print(f"Champs (Test) Events: {len(champs_events)}")

Total # Events: 193
Pre-Champs (Training) Events: 179
Champs (Test) Events: 14


In [4]:
pre_champs_matches = []
champs_matches = []
for match_list, event_list in ((pre_champs_matches, pre_champs_events), (champs_matches, champs_events)):
    for event in event_list:
        event_matches = tba.event_matches(event.key)
        for match in event_matches:
            if match.score_breakdown != None: # match has been finished & reported score
                match_list.append(match)

In [5]:
print(f"Pre-Champs (Training) Matches: {len(pre_champs_matches)}")
print(f"Champs (Test) Matches: {len(champs_matches)}")

Pre-Champs (Training) Matches: 16434
Champs (Test) Matches: 1588


In [6]:
all_features = list(pre_champs_matches[0].score_breakdown['red'].keys())
print(f"{len(all_features)} Features: {all_features}")
# split up between alliance features and individual features
robot_features = list(filter(lambda f : 'Robot' in f, all_features))
alliance_features = list(filter(lambda f : f not in robot_features, all_features))
print(f"Alliance features: {alliance_features}")
print(f"Robot features: {robot_features}")

51 Features: ['adjustPoints', 'autoPoints', 'bay1', 'bay2', 'bay3', 'bay4', 'bay5', 'bay6', 'bay7', 'bay8', 'cargoPoints', 'completeRocketRankingPoint', 'completedRocketFar', 'completedRocketNear', 'endgameRobot1', 'endgameRobot2', 'endgameRobot3', 'foulCount', 'foulPoints', 'habClimbPoints', 'habDockingRankingPoint', 'habLineRobot1', 'habLineRobot2', 'habLineRobot3', 'hatchPanelPoints', 'lowLeftRocketFar', 'lowLeftRocketNear', 'lowRightRocketFar', 'lowRightRocketNear', 'midLeftRocketFar', 'midLeftRocketNear', 'midRightRocketFar', 'midRightRocketNear', 'preMatchBay1', 'preMatchBay2', 'preMatchBay3', 'preMatchBay6', 'preMatchBay7', 'preMatchBay8', 'preMatchLevelRobot1', 'preMatchLevelRobot2', 'preMatchLevelRobot3', 'rp', 'sandStormBonusPoints', 'techFoulCount', 'teleopPoints', 'topLeftRocketFar', 'topLeftRocketNear', 'topRightRocketFar', 'topRightRocketNear', 'totalPoints']
Alliance features: ['adjustPoints', 'autoPoints', 'bay1', 'bay2', 'bay3', 'bay4', 'bay5', 'bay6', 'bay7', 'bay8', 

In [7]:
# choice starts with None, but is here's the guide:
# choice==str if feature is categorical
# choice==int if feature can be simply averaged
# choice==bool if feature is 0/1

feature_vals = {} # feature -> (choice, [list of values])
for feature in all_features:
    feature_choices = set([])
    # getting all values
    for match in pre_champs_matches + champs_matches:
        for alliance in ['red', 'blue']:
            feature_choices.add(match.score_breakdown[alliance][feature])
    feature_choices = sorted(list(feature_choices))
    # determining
    choice = str
    if all(map(lambda f: isinstance(f, bool), feature_choices)):
        choice = bool
    elif all(map(lambda f : isinstance(f, int), feature_choices)):
        choice = int
        
    feature_vals[feature] = (choice, feature_choices)
    print(f"{feature} ({choice}): {sorted(feature_choices)}")

adjustPoints (<class 'int'>): [-97, -90, -88, -85, -83, -82, -80, -78, -76, -74, -73, -72, -71, -70, -69, -68, -67, -65, -64, -63, -62, -61, -59, -57, -56, -53, -51, -50, -49, -47, -46, -36, -24, -8, -4, -3, -2, -1, 0, 1, 2, 3]
autoPoints (<class 'int'>): [0, 3, 6, 9, 12, 15, 18]
bay1 (<class 'str'>): ['None', 'Panel', 'PanelAndCargo']
bay2 (<class 'str'>): ['None', 'Panel', 'PanelAndCargo']
bay3 (<class 'str'>): ['None', 'Panel', 'PanelAndCargo']
bay4 (<class 'str'>): ['None', 'Panel', 'PanelAndCargo']
bay5 (<class 'str'>): ['None', 'Panel', 'PanelAndCargo']
bay6 (<class 'str'>): ['None', 'Panel', 'PanelAndCargo']
bay7 (<class 'str'>): ['None', 'Panel', 'PanelAndCargo']
bay8 (<class 'str'>): ['None', 'Panel', 'PanelAndCargo']
cargoPoints (<class 'int'>): [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60]
completeRocketRankingPoint (<class 'bool'>): [False, True]
completedRocketFar (<class 'bool'>): [False, True]
completedRocketNear (<class 'bool'>): [Fals

In [8]:
def getFeaturesRow(match, alliance, robot_num):
    features = []
    for feature in alliance_features + [f for f in robot_features if f"Robot{robot_num}" in f]:
        feature_corrected_name = feature[:-1] if feature[-6:-1] == 'Robot' else feature
        if feature_vals[feature][0] == int:
            features.append(feature_corrected_name)
        elif feature_vals[feature][0] == bool:
            features.append(feature_corrected_name)
        else: # feature_vals[feature][0] == str
            for val in feature_vals[feature][1]:
                features.append(f"{feature_corrected_name}_{val}")
    return features

def getData(match, alliance, robot_num):
    data = []
    for feature in alliance_features + [f for f in robot_features if f"Robot{robot_num}" in f]:
        if feature_vals[feature][0] == int:
            data.append(match.score_breakdown[alliance][feature])
        elif feature_vals[feature][0] == bool:
            data.append(1 if match.score_breakdown[alliance][feature] else 0)
        else: # feature_vals[feature][0] == str
            to_app = [0 for _ in range(len(feature_vals[feature][1]))]
            to_app[feature_vals[feature][1].index(match.score_breakdown[alliance][feature])] = 1
            data.extend(to_app)
    return data

In [9]:
args = {
    'match': champs_matches[0],
    'alliance': 'blue',
    'robot_num': 1,
}
features_row = getFeaturesRow(**args)
print(len(features_row), features_row)
data_row = getData(**args)
print("--")
print(len(data_row), data_row)

106 ['adjustPoints', 'autoPoints', 'bay1_None', 'bay1_Panel', 'bay1_PanelAndCargo', 'bay2_None', 'bay2_Panel', 'bay2_PanelAndCargo', 'bay3_None', 'bay3_Panel', 'bay3_PanelAndCargo', 'bay4_None', 'bay4_Panel', 'bay4_PanelAndCargo', 'bay5_None', 'bay5_Panel', 'bay5_PanelAndCargo', 'bay6_None', 'bay6_Panel', 'bay6_PanelAndCargo', 'bay7_None', 'bay7_Panel', 'bay7_PanelAndCargo', 'bay8_None', 'bay8_Panel', 'bay8_PanelAndCargo', 'cargoPoints', 'completeRocketRankingPoint', 'completedRocketFar', 'completedRocketNear', 'foulCount', 'foulPoints', 'habClimbPoints', 'habDockingRankingPoint', 'hatchPanelPoints', 'lowLeftRocketFar_None', 'lowLeftRocketFar_Panel', 'lowLeftRocketFar_PanelAndCargo', 'lowLeftRocketNear_None', 'lowLeftRocketNear_Panel', 'lowLeftRocketNear_PanelAndCargo', 'lowRightRocketFar_None', 'lowRightRocketFar_Panel', 'lowRightRocketFar_PanelAndCargo', 'lowRightRocketNear_None', 'lowRightRocketNear_Panel', 'lowRightRocketNear_PanelAndCargo', 'midLeftRocketFar_None', 'midLeftRocketF

In [10]:
writer = open(output_file, 'w')
all_col_names = ["preChamps", "match_key", "team"] + features_row + ["winner"]
writer.write(",".join(all_col_names) + "\n")
for preChamps, match_list in ((1, pre_champs_matches), (0, champs_matches)):
    for match in match_list:
        for alliance in ('red', 'blue'):
            for robot in [1, 2, 3]:
                all_row_feats = map(str, [preChamps, match.key, match.alliances[alliance]['team_keys'][robot - 1]] + getData(match, alliance, robot) + [0 if winning_alliance])
                writer.write(",".join(all_row_feats) + "\n")
writer.close()

AttributeError: 'Match' object has no attribute 'winner'

In [11]:
champs_matches[0]

Match({'actual_time': 1555781327, 'alliances': {'blue': {'dq_team_keys': [], 'score': 106, 'surrogate_team_keys': [], 'team_keys': ['frc1678', 'frc3132', 'frc7179']}, 'red': {'dq_team_keys': [], 'score': 93, 'surrogate_team_keys': [], 'team_keys': ['frc1746', 'frc6443', 'frc7426']}}, 'comp_level': 'f', 'event_key': '2019carv', 'key': '2019carv_f1m1', 'match_number': 1, 'post_result_time': 1555782842, 'predicted_time': 1555782606, 'score_breakdown': {'blue': {'adjustPoints': 0, 'autoPoints': 15, 'bay1': 'PanelAndCargo', 'bay2': 'PanelAndCargo', 'bay3': 'PanelAndCargo', 'bay4': 'PanelAndCargo', 'bay5': 'PanelAndCargo', 'bay6': 'PanelAndCargo', 'bay7': 'PanelAndCargo', 'bay8': 'Panel', 'cargoPoints': 33, 'completeRocketRankingPoint': False, 'completedRocketFar': False, 'completedRocketNear': False, 'endgameRobot1': 'HabLevel3', 'endgameRobot2': 'HabLevel3', 'endgameRobot3': 'HabLevel3', 'foulCount': 1, 'foulPoints': 0, 'habClimbPoints': 36, 'habDockingRankingPoint': False, 'habLineRobot1'

## Simple Output

In [ ]:
writer = open(output_file2, 'w')
all_col_names = ["preChamps", "match_key", "r1", "r2", "r3", "b1", "b2", "b3", "winner"]
writer.write(",".join(all_col_names) + "\n")
for preChamps, match_list in ((1, pre_champs_matches), (0, champs_matches)):
    for match in match_list:
        output_data = [preChamps, match.key] + match.alliances['red']['team_keys'][:3] + match.alliances['blue']['team_keys'][:3] + [0 if match.winning_alliance == 'red' else 1]
        writer.write(",".join([str(a) for a in output_data]) + "\n")